### Create a Posteoperative Patient Data Predictive Model

#### Packages

In [0]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import when, col
from pyspark.ml.feature import VectorAssembler

In [0]:
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.jobs import Task, NotebookTask, Source

#### Mount My Storage to db

In [0]:
storage_account_name = "mlrstorage2025"
container_name = "my-container"
mount_point = "/mnt/postoperative_data"
sas_token = "sp=r&st=2025-03-26T05:37:45Z&se=2025-03-26T13:37:45Z&spr=https&sv=2024-11-04&sr=c&sig=bU4ljnmAWDEJyDp2gc2DVPCaFFQbxcfOjicy5sFeLcQ%3D"

configs = {
    f"fs.azure.sas.{container_name}.{storage_account_name}.blob.core.windows.net": sas_token
}

dbutils.fs.mount(
  source = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net",
  mount_point = mount_point,
  extra_configs = configs
)

True

In [0]:
dbutils.fs.mounts()

[MountInfo(mountPoint='/databricks-datasets', source='databricks-datasets', encryptionType=''),
 MountInfo(mountPoint='/Volumes', source='UnityCatalogVolumes', encryptionType=''),
 MountInfo(mountPoint='/databricks/mlflow-tracking', source='databricks/mlflow-tracking', encryptionType=''),
 MountInfo(mountPoint='/databricks-results', source='databricks-results', encryptionType=''),
 MountInfo(mountPoint='/databricks/mlflow-registry', source='databricks/mlflow-registry', encryptionType=''),
 MountInfo(mountPoint='/mnt/postoperative_data', source='wasbs://my-container@mlrstorage2025.blob.core.windows.net', encryptionType=''),
 MountInfo(mountPoint='/Volume', source='DbfsReserved', encryptionType=''),
 MountInfo(mountPoint='/volumes', source='DbfsReserved', encryptionType=''),
 MountInfo(mountPoint='/', source='DatabricksRoot', encryptionType=''),
 MountInfo(mountPoint='/volume', source='DbfsReserved', encryptionType='')]

In [0]:
file_path = "/mnt/postoperative_data/postoperative_data.parquet"
df = spark.read.parquet(file_path)

In [0]:
display(df)

L-CORE,L-SURF,L-O2,L-BP,SURF-STBL,CORE-STBL,BP-STBL,COMFORT,decision
mid,low,excellent,mid,stable,stable,stable,15,A
mid,high,excellent,high,stable,stable,stable,10,S
high,low,excellent,high,stable,stable,mod-stable,10,A
mid,low,good,high,stable,unstable,mod-stable,15,A
mid,mid,excellent,high,stable,stable,stable,10,A
high,low,good,mid,stable,stable,unstable,15,S
mid,low,excellent,high,stable,stable,mod-stable,05,S
high,mid,excellent,mid,unstable,unstable,stable,10,S
mid,high,good,mid,stable,stable,stable,10,S
mid,low,excellent,mid,unstable,stable,mod-stable,10,S


#### Process Data using Spark

In [0]:
df = df.withColumn("index", monotonically_increasing_id())
df = df.select("index", "*")
display(df)

index,L-CORE,L-SURF,L-O2,L-BP,SURF-STBL,CORE-STBL,BP-STBL,COMFORT,decision,index
0,mid,low,excellent,mid,stable,stable,stable,15,A,0
1,mid,high,excellent,high,stable,stable,stable,10,S,1
2,high,low,excellent,high,stable,stable,mod-stable,10,A,2
3,mid,low,good,high,stable,unstable,mod-stable,15,A,3
4,mid,mid,excellent,high,stable,stable,stable,10,A,4
5,high,low,good,mid,stable,stable,unstable,15,S,5
6,mid,low,excellent,high,stable,stable,mod-stable,05,S,6
7,high,mid,excellent,mid,unstable,unstable,stable,10,S,7
8,mid,high,good,mid,stable,stable,stable,10,S,8
9,mid,low,excellent,mid,unstable,stable,mod-stable,10,S,9


In [0]:
# Categorial feature transformation
indexer = StringIndexer(inputCol="L-CORE", outputCol="L-CORE_index")
pipeline = Pipeline(stages=[indexer])
df = pipeline.fit(df).transform(df)
display(df)

index,L-CORE,L-SURF,L-O2,L-BP,SURF-STBL,CORE-STBL,BP-STBL,COMFORT,decision,index,L-CORE_index
0,mid,low,excellent,mid,stable,stable,stable,15,A,0,0.0
1,mid,high,excellent,high,stable,stable,stable,10,S,1,0.0
2,high,low,excellent,high,stable,stable,mod-stable,10,A,2,2.0
3,mid,low,good,high,stable,unstable,mod-stable,15,A,3,0.0
4,mid,mid,excellent,high,stable,stable,stable,10,A,4,0.0
5,high,low,good,mid,stable,stable,unstable,15,S,5,2.0
6,mid,low,excellent,high,stable,stable,mod-stable,05,S,6,0.0
7,high,mid,excellent,mid,unstable,unstable,stable,10,S,7,2.0
8,mid,high,good,mid,stable,stable,stable,10,S,8,0.0
9,mid,low,excellent,mid,unstable,stable,mod-stable,10,S,9,0.0


In [0]:
df = df.withColumn(
    "COMFORT_Category",
    when(col("COMFORT").between(0, 5), "Very Uncomfortable")
    .when(col("COMFORT").between(6, 10), "Uncomfortable")
    .when(col("COMFORT").between(11, 15), "Comfortable")
    .when(col("COMFORT").between(16, 20), "Very Comfortable")
    .otherwise("Unknown")
)

display(df)

index,L-CORE,L-SURF,L-O2,L-BP,SURF-STBL,CORE-STBL,BP-STBL,COMFORT,decision,index,L-CORE_index,COMFORT_Category
0,mid,low,excellent,mid,stable,stable,stable,15,A,0,0.0,Comfortable
1,mid,high,excellent,high,stable,stable,stable,10,S,1,0.0,Uncomfortable
2,high,low,excellent,high,stable,stable,mod-stable,10,A,2,2.0,Uncomfortable
3,mid,low,good,high,stable,unstable,mod-stable,15,A,3,0.0,Comfortable
4,mid,mid,excellent,high,stable,stable,stable,10,A,4,0.0,Uncomfortable
5,high,low,good,mid,stable,stable,unstable,15,S,5,2.0,Comfortable
6,mid,low,excellent,high,stable,stable,mod-stable,05,S,6,0.0,Very Uncomfortable
7,high,mid,excellent,mid,unstable,unstable,stable,10,S,7,2.0,Uncomfortable
8,mid,high,good,mid,stable,stable,stable,10,S,8,0.0,Uncomfortable
9,mid,low,excellent,mid,unstable,stable,mod-stable,10,S,9,0.0,Uncomfortable


In [0]:
column_mapping = {
    ('L-CORE', 'L_CORE'), 
    ('L-SURF', 'L_SURF'), 
    ('L-O2', 'L_O2'), 
    ('L-BP', 'L_BP'), 
    ('SURF-STBL', 'SURF_STBL'), 
    ('CORE-STBL', 'CORE_STBL'), 
    ('BP-STBL', 'BP_STBL'),
    ('decision', 'discharge_decision')
}

for old_name, new_name in column_mapping:
    df = df.withColumnRenamed(old_name, new_name)

display(df)

index,L_CORE,L_SURF,L_O2,L_BP,SURF_STBL,CORE_STBL,BP_STBL,COMFORT,discharge_decision,index,L-CORE_index,COMFORT_Category
0,mid,low,excellent,mid,stable,stable,stable,15,A,0,0.0,Comfortable
1,mid,high,excellent,high,stable,stable,stable,10,S,1,0.0,Uncomfortable
2,high,low,excellent,high,stable,stable,mod-stable,10,A,2,2.0,Uncomfortable
3,mid,low,good,high,stable,unstable,mod-stable,15,A,3,0.0,Comfortable
4,mid,mid,excellent,high,stable,stable,stable,10,A,4,0.0,Uncomfortable
5,high,low,good,mid,stable,stable,unstable,15,S,5,2.0,Comfortable
6,mid,low,excellent,high,stable,stable,mod-stable,05,S,6,0.0,Very Uncomfortable
7,high,mid,excellent,mid,unstable,unstable,stable,10,S,7,2.0,Uncomfortable
8,mid,high,good,mid,stable,stable,stable,10,S,8,0.0,Uncomfortable
9,mid,low,excellent,mid,unstable,stable,mod-stable,10,S,9,0.0,Uncomfortable


#### Train Model

In [0]:
feature_columns = ["L_CORE","L_SURF","L_O2","L_BP","SURF_STBL","CORE_STBL","BP_STBL","COMFORT","discharge_decision","COMFORT_Category"]

In [0]:
indexers = [StringIndexer(inputCol=col, outputCol=col + "_index") for col in feature_columns]
encoders = [OneHotEncoder(inputCol=col + "_index", outputCol=col + "_encoded") for col in feature_columns]

for indexer, encoder in zip(indexers, encoders):
    df = indexer.fit(df).transform(df)
    df = encoder.fit(df).transform(df)

encoded_columns = [col + "_encoded" for col in feature_columns]
feature_columns = [col for col in feature_columns if col not in feature_columns] + encoded_columns

assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
df = assembler.transform(df)

display(df)

index,L_CORE,L_SURF,L_O2,L_BP,SURF_STBL,CORE_STBL,BP_STBL,COMFORT,discharge_decision,index,L-CORE_index,COMFORT_Category,L_CORE_index,L_CORE_encoded,L_SURF_index,L_SURF_encoded,L_O2_index,L_O2_encoded,L_BP_index,L_BP_encoded,SURF_STBL_index,SURF_STBL_encoded,CORE_STBL_index,CORE_STBL_encoded,BP_STBL_index,BP_STBL_encoded,COMFORT_index,COMFORT_encoded,discharge_decision_index,discharge_decision_encoded,COMFORT_Category_index,COMFORT_Category_encoded,features
0,mid,low,excellent,mid,stable,stable,stable,15,A,0,0.0,Comfortable,0.0,"Map(vectorType -> sparse, length -> 2, indices -> List(0), values -> List(1.0))",1.0,"Map(vectorType -> sparse, length -> 2, indices -> List(1), values -> List(1.0))",1.0,"Map(vectorType -> sparse, length -> 1, indices -> List(), values -> List())",0.0,"Map(vectorType -> sparse, length -> 2, indices -> List(0), values -> List(1.0))",0.0,"Map(vectorType -> sparse, length -> 1, indices -> List(0), values -> List(1.0))",0.0,"Map(vectorType -> sparse, length -> 2, indices -> List(0), values -> List(1.0))",0.0,"Map(vectorType -> sparse, length -> 2, indices -> List(0), values -> List(1.0))",1.0,"Map(vectorType -> sparse, length -> 4, indices -> List(1), values -> List(1.0))",0.0,"Map(vectorType -> sparse, length -> 2, indices -> List(0), values -> List(1.0))",1.0,"Map(vectorType -> sparse, length -> 3, indices -> List(1), values -> List(1.0))","Map(vectorType -> sparse, length -> 21, indices -> List(0, 3, 5, 7, 8, 10, 13, 16, 19), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0))"
1,mid,high,excellent,high,stable,stable,stable,10,S,1,0.0,Uncomfortable,0.0,"Map(vectorType -> sparse, length -> 2, indices -> List(0), values -> List(1.0))",2.0,"Map(vectorType -> sparse, length -> 2, indices -> List(), values -> List())",1.0,"Map(vectorType -> sparse, length -> 1, indices -> List(), values -> List())",1.0,"Map(vectorType -> sparse, length -> 2, indices -> List(1), values -> List(1.0))",0.0,"Map(vectorType -> sparse, length -> 1, indices -> List(0), values -> List(1.0))",0.0,"Map(vectorType -> sparse, length -> 2, indices -> List(0), values -> List(1.0))",0.0,"Map(vectorType -> sparse, length -> 2, indices -> List(0), values -> List(1.0))",0.0,"Map(vectorType -> sparse, length -> 4, indices -> List(0), values -> List(1.0))",1.0,"Map(vectorType -> sparse, length -> 2, indices -> List(1), values -> List(1.0))",0.0,"Map(vectorType -> sparse, length -> 3, indices -> List(0), values -> List(1.0))","Map(vectorType -> sparse, length -> 21, indices -> List(0, 6, 7, 8, 10, 12, 17, 18), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0))"
2,high,low,excellent,high,stable,stable,mod-stable,10,A,2,2.0,Uncomfortable,2.0,"Map(vectorType -> sparse, length -> 2, indices -> List(), values -> List())",1.0,"Map(vectorType -> sparse, length -> 2, indices -> List(1), values -> List(1.0))",1.0,"Map(vectorType -> sparse, length -> 1, indices -> List(), values -> List())",1.0,"Map(vectorType -> sparse, length -> 2, indices -> List(1), values -> List(1.0))",0.0,"Map(vectorType -> sparse, length -> 1, indices -> List(0), values -> List(1.0))",0.0,"Map(vectorType -> sparse, length -> 2, indices -> List(0), values -> List(1.0))",2.0,"Map(vectorType -> sparse, length -> 2, indices -> List(), values -> List())",0.0,"Map(vectorType -> sparse, length -> 4, indices -> List(0), values -> List(1.0))",0.0,"Map(vectorType -> sparse, length -> 2, indices -> List(0), values -> List(1.0))",0.0,"Map(vectorType -> sparse, length -> 3, indices -> List(0), values -> List(1.0))","Map(vectorType -> sparse, length -> 21, indices -> List(3, 6, 7, 8, 12, 16, 18), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0))"
3,mid,low,good,high,stable,unstable,mod-stable,15,A,3,0.0,Comfortable,0.0,"Map(vectorType -> sparse, length -> 2, indices -> List(0), values -> List(1.0))",1.0,"Map(vectorType -> sparse, length -> 2, indices -> List(1), values -> List(1.0))",0.0,"Map(vectorType -> sparse, length -> 1, indices -> List(0), values -> List(1.0))",1.0,"Map(v

In [0]:
# Split the data
train_data, test_data = df.randomSplit([0.8, 0.2], seed=42)

# Train the model
classifier = RandomForestClassifier(labelCol="discharge_decision_index", featuresCol="features")
model = classifier.fit(train_data)

predictions = model.transform(test_data)

# Evaluate the model
evaluator = MulticlassClassificationEvaluator(labelCol="discharge_decision_index", predictionCol="prediction")
accuracy = evaluator.evaluate(predictions)

print(f"Model accuracy: {accuracy}")

Model accuracy: 0.9129411764705884


#### Store the Processed Data

In [0]:
df_without_index = df.drop("index")

storage_account_name = "mlrstorage2025"
container_name = "my-container"
sas_token = "sp=racwdli&st=2025-03-26T08:26:02Z&se=2025-03-26T16:26:02Z&sv=2024-11-04&sr=c&sig=obreMqDEGcyCj1BsJhDC8YXxT7jEPqmslPg6sZj4krw%3D"

output_path = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/your/output/path/"

# Configure Spark session with the SAS token
spark.conf.set(
    f"fs.azure.sas.{container_name}.{storage_account_name}.blob.core.windows.net",
    sas_token
)

try:
    df_without_index.write.mode("overwrite").parquet(output_path)
    print(f"Successfully wrote DataFrame to: {output_path}")
except Exception as e:
    print(f"An error occurred while writing to Parquet: {e}")

Successfully wrote DataFrame to: wasbs://my-container@mlrstorage2025.blob.core.windows.net/your/output/path/
